In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd

train_df = pd.read_csv('../../data/python_data/ratings_train.txt',sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [3]:
# 학습 데이터의 레이블
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [4]:
# document 5건 null이 있음
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [5]:
import re

train_df = train_df.fillna(' ')
# re - 숫자를 공백으로 변환
train_df['document'] = train_df['document'].apply(lambda x:re.sub(r"\d+"," ",x))

# re - Null 및 숫자를 공백으로 변환
# \d+ - 1개 이상의 숫자가 나오면 공백으로 변경
test_df = pd.read_csv('../../data/python_data/ratings_test.txt',sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x:re.sub(r"\d+"," ",x))

# id 컬럼 삭제
train_df.drop('id',axis=1,inplace=True)
test_df.drop('id',axis=1,inplace=True)

In [6]:
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
    # 입력 들어온 text를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

tw_tokenizer('오늘은 날씨가 좋습니다.')

['오늘', '은', '날씨', '가', '좋습니다', '.']

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# ngram_range = (1,2) : go back, go home
# ngram_range = (1,1) : go, back, home
# min_df = 3 : 최소 빈도값을 설정, df = 문서의 수
# max_df = 0.9 : 90% 이상 나타나는 단어는 무시, 10개의 문장에서 9번 이상 나오는 단어는 무시
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer,ngram_range=(1,2),min_df=3,max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [13]:
print(tfidf_matrix_train.shape)

(150000, 129276)


In [12]:
# Logistic Regression 이용 감성 분석 Classification 수행
lg_clf = LogisticRegression(random_state=0)

# Parameter C 최적화 - GridSearchCV
params = {'C':[1,3.5,4.5,5.5,10]}
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3,scoring='accuracy',verbose=1)

grid_cv.fit(tfidf_matrix_train,train_df['label'])
print(grid_cv.best_params_,round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 4.5} 0.8592


In [14]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer을 이용
# 테스트 데이터를 TF-IDF 값으로 Feature 변환
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# 최적 파라미터로 학습된 classifier를 그대로 사용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도 : ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도 :  0.86178


In [15]:
test_df['document'][150]

'다시 봐도 수작이다. 미국 국수주의라 뭐라는 시점도 있겠으나 전쟁물 중에 손가락안에 꼽을정도로 잘만들어 졌다. 스토리 라인은 뻔하나 그 뻔한걸 너무 잘 살렸다. 언제 봐도 다시봐도 재미 있다.  년이 지난 영화가..'

In [16]:
best_estimator.predict(tfidf_vect.transform([test_df['document'][150]]))

array([1], dtype=int64)

In [17]:
test_df['document'][2]

'뭐야 이 평점들은.... 나쁘진 않지만  점 짜리는 더더욱 아니잖아'

In [18]:
best_estimator.predict(tfidf_vect.transform([test_df['document'][2]]))

array([0], dtype=int64)

In [22]:
text = '영화 재미없네요'

if best_estimator.predict(tfidf_vect.transform([text])) == 0:
    print(f'"{text}"-> 부정일 가능성이 {round(best_estimator.predict_proba(tfidf_vect.transform([text]))[0][0],2)}% 입니다.')
    
else:
    print(f'"{text}"-> 긍정일 가능성이 {round(best_estimator.predict_proba(tfidf_vect.transform([text]))[0][1],2)}% 입니다.')

"영화 재미없네요"-> 부정일 가능성이 1.0% 입니다.


In [23]:
text = '최고의 액션 영화입니다'

if best_estimator.predict(tfidf_vect.transform([text])) == 0:
    print(f'"{text}"-> 부정일 가능성이 {round(best_estimator.predict_proba(tfidf_vect.transform([text]))[0][0],2)}% 입니다.')
    
else:
    print(f'"{text}"-> 긍정일 가능성이 {round(best_estimator.predict_proba(tfidf_vect.transform([text]))[0][1],2)}% 입니다.')

"최고의 액션 영화입니다"-> 긍정일 가능성이 0.99% 입니다.


In [24]:
result = pd.DataFrame({
    "document":test_df['document'],
    "test":test_df['label'],
    "pred":preds
})

In [28]:
result.tail()
# 실제는 부정인데 긍정으로 예측한 부분이 있음(49997)

,document,test,pred
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,0
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,1
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,0
49999,마무리는 또 왜이래,0,0
